# Let's begin

In [1]:
#import tools for twitter API
import tweepy
from twitter import *
import pymongo, pprint
from pymongo import MongoClient
#Import everything useful for networks
import networkx as nx
import matplotlib.pyplot as plt
import json
import time
import datetime
import os
import pickle
import elections_helper as helper

### Twitter APIs

In [2]:
properties = dict(line.strip().split('=') 
          for line in open('./properties/elections.properties') 
          if not line.startswith('#') and not line.startswith('\n'))

In [3]:
CONSUMER_KEY = 'aF03g35xEjOm5OQuFw8PSuwLF'
CONSUMER_SECRET ='5Ifz6t0UPa6BxaaVm7eTQF4cOe3XcpVsX535nFkaLqy5bp7ehg'
OAUTH_TOKEN = '594338908-f0JBTAwwVGoZErJ4f1YNAgkj2gws7FQaFur4r2gs'
OAUTH_TOKEN_SECRET = 'EOR9ptKEtAc0LyhsEOAxsyEshWdgTcJKzXS6iBln2iB6h'

In [4]:
# load our API credentials 
twitter = Twitter(
    auth = OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET, CONSUMER_KEY, CONSUMER_SECRET))

In [5]:
#Credentials for tweepy (use it as complement)
auth = tweepy.OAuthHandler(properties["CONSUMER_KEY"], properties["CONSUMER_SECRET"])
auth.set_access_token(properties["OAUTH_TOKEN"], properties["OAUTH_TOKEN_SECRET"])
api = tweepy.API(auth)

### Get the users from Mongo DB

For more efficiency we used Mongo data base to store all of our datas so they are available by anyone at anytime.

In [6]:
# Connect to MongoDB
client = helper.setup_mongo_client(properties_file='./properties/db.properties')

_, user_collection = helper.get_collections(client)

print "Number of users %s" % user_collection.count()

Number of users 87501


### Create the network

Now that everything is settled we can start working on our tweet. 

In [7]:
# Create a list of user_ids
all_user_ids = dict([(user['_id'], 1) for user in user_collection.find()])

print "Number of users %s" % len(all_user_ids)

Number of users 87501


In [38]:
# Create the graph
directed = nx.DiGraph()

# All twitter users in the database
users = user_collection.find()

for i in users:
    
    _id = i["_id"]
    # We need to deal with the rate limit of twitter API
    # It's not the best way but it works and we don't really need more
    try:
        #select the user name
        user = api.get_user(_id)
        username = user.screen_name

        # perform a basic search 
        query = twitter.friends.ids(screen_name = username)
                
        found_ids = [user_id for user_id in query["ids"] if all_user_ids.get(user_id, 0) != 0]

        print "found %d friends but only %s tweeted about the election" % (len(query["ids"]),len(found_ids))
        
        if len(found_ids) == 0:
            continue
            
        # now we loop through them to pull out more info, in blocks of 100.
        for n in range(0, len(found_ids), 100):
            ids = found_ids[n:n+100]
            
            # create a subquery, looking up information about these users
            subquery = twitter.users.lookup(user_id = ids)

            for user_q in subquery:
                # now print out user info, starring any users that are Verified.
                directed.add_node(user_q["screen_name"])
                directed.add_edge(username, user_q["screen_name"])
                #print " [%s] %s" % ("*" if user["verified"] else " ", user["screen_name"])
    except Exception as e:
        
        dct = e.__dict__
        
        if dct.get('response_data',0) == 0:
            message = dct
            code = dct['api_code']
        else:
            message = dct['response_data']['errors'][0]
            code = message['code']
        print "Message: %s" % message
        
        if  code == 88:
            #Sleep while we cannot access twitter
            time.sleep(15 * 60 + 15)
        else:
            print "User id: %s" % _id
            print "User: %s" % i
            continue
        continue

In [41]:
# Save the network
pickle.dump(directed, open('graph2.txt', 'w'))